<a href="https://colab.research.google.com/github/marissa-graham/multiagent_systems/blob/master/Linear_Programming_for_Nash_Equilibria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

Use a linear programming solver to find the Nash equilibrium for the following games:

* Battle of the Sexes (page 58-59 of text).
* Rock, Paper Scissors (pages 57-58 of the text).
* A 3x3 zero-sum game that you design. The solution to this game must be a mixed strategy and the payoff matrix cannot be symmetric.

Do your solutions make sense? Justify that the solutions are indeed equilbria for the zero-sum games, and discuss whether the solution returned by your program is correct for the Battle of the Sexes game.

## Email notes

 At least one of the games isn't zero sum. The solver will return a maximin solution -- that is what it does -- but solution pair may produced by the two maximin solutions may not be a Nash equilibrium.

You can check whether the pair is a Nash equilibrium without knowing what the actual Nash equilibria are. See if either player has a unilateral incentive to change from the pair of maximin solutions. If one does, then the solution is not a Nash equilibrium.

# Installing and importing modules

In [0]:
!pip install cvxopt

    100% |████████████████████████████████| 11.6MB 2.6MB/s 


In [0]:
from cvxopt import matrix, solvers
import numpy as np

# Setting up solver

### Notes about cvxopt and setup




cvxopt is a Python module for convex optimization

(Fun fact we used it in ACME like four years ago and I didn't realize 'cvx' was an abbreviation for 'convex' until Dr. Evans told me so early this week)

You set up a cvxopt matrix object like a numpy array. For the linear program

$$ \text{minimize } c^Tx$$
$$ \text{subject to } Gx\leq h, Ax = b, $$

you call `solvers.lp(c,G,h,A,b)` to get the solution `sol` and access the solution as `sol['x']`.

How do we turn that into this?

$$ \text{minimize } u_1^* $$
$$ \text{subject to } \sum_{k\in A_2} u_1(a_1^j,a_2^k)s_2^k \leq u_1^* \forall j\in A_1 $$
$$\sum_{k\in A_2} s_2^k = 1, s_2^k\geq 0. $$

* Let $n$ be the number of actions.

* $x$ is $u_1^*$ tacked onto the vector of $s_2^k$'s, which is length $n$. So, $A$ is a (length $n+1$) vector of ones except for the first entry, which has a zero, appropriately shaped to get a dot product, and $b$ is just a 1x1 with a 1 in it. 

* $c$ is a vector with a 1 tacked onto a zero vector of length $n$ (total length $n+1$)

* To get the nonnegativity condition, we'll tack a negative identity matrix (only over on the right hand side, it can skip $u_1^*$, so it only takes up $n$ rows) onto the bottom of $G$ and a corresponding bunch of zeros onto the bottom of $h$. 

* Let's subtract $u_1^*$ over, since we're multiplying by it, so $h$ is just a vector of zeros. Length $2n$. 

* The hard(est) part is then the bit where we figure out how to set up the top part of $G$. It'll be based on the game matrix, and we get one row for each action, because of that $\forall j$ condition. We know $G$ is going to be $2n \times n$.

Constructing the first $n$ rows of $G$:

* The first column is just all -1, since we subtracted over the $u_1^*$.
* Fix a value of $j$. This will be the $j$-th row. We want it to be utilities of player 1 playing his or her $j$-th action for each action of player 2. This is just the $j$-th row of the game matrix slice for player 1. So we just put that whole thing into the top right corner of $G$. Wow. That's really, really easy.

### Actual code

In [0]:
class NormalFormGame:
    """
    Work with games in normal form given a 3-D payoff matrix.
    
    M is the game matrix, as a numpy array.
    
    - If n is the number of actions, its shape will be n x n x 2. 
    - M[:,:,0] is the payoff matrix for Player 1
    - M[:,:,1] is the payoff matrix for Player 2
    - The rows are indexed by the actions of Player 1
    - The columns are indexed by the actions of Player 2
    """
    def __init__(self, M):
        
        self.M = M
        
        if self.is_zero_sum() == False:
            print("WARNING: NOT A ZERO SUM GAME.")
            
        self.p1 = self.solve(M)
        self.p2 = self.solve(M[:,:,[1,0]])
        print("Player 1 Solution:", self.p1)
        print("Player 2 Solution:", self.p2)
        print("Nash equilibrium?", self.is_Nash_equilibrium())
    
    def solve(self, M):
        
        n = M.shape[0]
        b = matrix([1.0])
        h = matrix(np.zeros(2*n))
        
        pre_A = np.ones(n+1)
        pre_A[0] = 0.0
        A = matrix(pre_A.reshape((1,n+1)))
        
        pre_c = np.zeros(n+1)
        pre_c[0] = 1.0
        c = matrix(pre_c)
        
        pre_G = np.zeros((2*n,n+1))
        pre_G[n:,1:] = -1.0*np.eye(n) # Nonnegativity of s_k's
        pre_G[:n,0] = -1.0*np.ones(n) # Subtracted over the u1*'s
        pre_G[:n,1:] = M[:,:,0] # Utilities for player 1
        G = matrix(pre_G)
        
        s = solvers.lp(c,G,h,A,b,solver='glpk',
                       options={'glpk':{'msg_lev':'GLP_MSG_OFF'}})
        return s['x']
    
    def is_zero_sum(self):
        """Check if it's a zero sum game."""
        return np.all(np.sum(self.M,axis=2) == np.zeros(self.M.shape[:-1]))
        
    def is_Nash_equilibrium(self):
        """
        Check if either player has a unilateral incentive to change.
        
        They'll have a unilateral incentive to change if there is an action
        that gives them higher utility with respect to the other player's 
        strategy.
        """
        
        u1 = np.max(np.dot(self.M[:,:,0], self.p2[1:]))
        u2 = np.max(np.dot(self.M[:,:,1].T, self.p1[1:]))
        
        # Not requiring a strict Nash equilibrium
        if self.p1[0] >= u1 and self.p2[0] >= u2:
            return True
        
        else:
            return False

# Use the solver and explain results

## Set up the game matrices

Create the game matrices and make sure they're the right shape. Also figure out the right way to switch the players easily.

In [0]:
rps = np.array([[[0,0],[-1,1],[1,-1]],
                [[1,-1],[0,0],[-1,1]],
                [[-1,1],[1,-1],[0,0]]],dtype=np.float)
print(rps[:,:,0])
print(rps[:,:,[1,0]].shape) # Switch the order of the players

bs = np.array([[[2,1],[0,0]],
               [[0,0],[1,2]]],dtype=np.float)

# Player 1 benefits from coordinated action, Player 2 prefers mixed.
# Player 2 has a mild incentive to play action 3, but won't much, because 
# then player 1 can play action 3 and player 2 will get the worst case.
mine = np.array([[[2,-2],[-1,1],[-2,2]],
                [[-1,1],[2,-2],[-2,2]],
                [[-1,1],[-1,1],[4,-4]]],dtype=np.float)

[[ 0. -1.  1.]
 [ 1.  0. -1.]
 [-1.  1.  0.]]
(3, 3, 2)


## Run the solver for each game and check if it's a Nash equilibrium

### Battle of the Sexes

Since it's not a zero sum game, the solver only gives us a minimax solution for each player. Each player does best by favoring the other's preferred restaurant, but still choosing their own a third time.

This not a Nash equilibrium, since if your spouse is choosing your preferred restaurant 2/3rds of the time, you clearly benefit from choosing it more often.

In [0]:
BS_1 = NormalFormGame(bs)

Player 1 Solution: [ 6.67e-01]
[ 3.33e-01]
[ 6.67e-01]

Player 2 Solution: [ 6.67e-01]
[ 6.67e-01]
[ 3.33e-01]

Nash equilibrium? False


### Rock Paper Scissors

This is a zero sum game, and the Nash equilibrium is simply to play uniformly randomly. If either player adjusted their strategy to favor a certain option, the other player would be able to adjust their solution to exploit that, incentivizing the first player to adjust accordingly, etc.; we would not have an equilibrium.

In [0]:
RPS = NormalFormGame(rps)

Player 1 Solution: [ 0.00e+00]
[ 3.33e-01]
[ 3.33e-01]
[ 3.33e-01]

Player 2 Solution: [ 0.00e+00]
[ 3.33e-01]
[ 3.33e-01]
[ 3.33e-01]

Nash equilibrium? True


### My game

I made a game that favored coordinated actions for Player 1, and mixed for Player 2, favoring action 3 for Player 2 somewhat, but significantly favoring "both play action 3" for Player 1, disincentivizing Player 2 to play it. I also made sure the game was not biased towards either player

As a result, neither player favors action 3, and instead favors a mix of actions 1 and 2. This isn't a Nash equilibrium, since if Player 2 favors action 3, Player 1 can exploit this by favoring it more, but the more Player 1 favors action 3, the less Player 2 will play action 3, disincentivizing Player 1 to play action 3.

I assumed by "symmetric" you meant with respect to the payoff matrix for each player. It seems like it'd be difficult to find a Nash equilibrium for this game, but it's also possible I'm not checking for them correctly. I have to turn it in right now either way, though, because I procrastinated this and I need to catch the 10:47 train if I want to get home before midnight, but there's no wifi on the train to submit it.

In [0]:
mygame = NormalFormGame(mine)

Player 1 Solution: [-4.26e-02]
[ 2.55e-01]
[ 4.26e-01]
[ 3.19e-01]

Player 2 Solution: [ 4.26e-02]
[ 2.55e-01]
[ 4.26e-01]
[ 3.19e-01]

Nash equilibrium? False
